In [2]:
#  https://www.kaggle.com/code/khanh123987/tabular-question-answering

In [3]:
import torch

if torch.cuda.is_available():
    print("GPU 已开启")
    device = torch.device("cuda")
else:
    print("GPU 未开启")
    device = torch.device("cpu")

GPU 已开启


In [4]:
#指定设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
!pip install transformers --quiet
!pip install datasets --quiet
!pip install sentencepiece --quiet
!pip install accelerate --quiet
!pip install -U nltk
!pip install sacrebleu --quiet
!pip install bert_score --quiet
!pip install git+https://github.com/google-research/bleurt.git --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.2 MB/s eta 0:00:0000:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


transformers：用于自然语言处理和深度学习模型的Python库，包括预训练的语言模型（如BERT、GPT等）和用于文本分类、命名实体识别、问答等任务的工具。

datasets：提供了一个统一的API来加载和处理各种自然语言处理数据集。

sentencepiece：用于训练和使用字节对编码（Byte Pair Encoding，BPE）模型的工具。

accelerate：提供了加速深度学习模型训练的工具和框架，可利用多个GPU或TPU进行并行计算。

nltk：自然语言工具包（Natural Language Toolkit），提供了各种文本处理和分析的功能。

sacrebleu：用于计算机器翻译结果的BLEU评估指标的Python库。

bert_score：计算BERT模型生成的句子相似性得分的Python库。

bleurt：谷歌研究团队开发的用于评估文本生成质量的BERT模型。

这些包的安装将为后续的代码执行提供必要的依赖项和工具。

--quiet参数用于在安装过程中隐藏输出信息

In [6]:
import numpy as np
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.utils.data import DataLoader
import nltk
nltk.download("punkt", quiet=True)

import pandas as pd
from datasets import load_dataset, load_metric

import transformers
from transformers import (
    AutoConfig,
    BartForConditionalGeneration,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    TapexTokenizer,
    set_seed,
)
from transformers.trainer_utils import (
    get_last_checkpoint, 
    is_main_process, 
    default_compute_objective,
    # default_hp_space_ray
)

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['WANDB_DISABLED'] = 'true'

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


numpy：Python中处理数值计算的库。

torch：PyTorch深度学习框架的Python库。

os：用于与操作系统进行交互的Python模块。

torch.nn：PyTorch提供的神经网络模块。

math：Python提供的数学运算模块。

DataLoader：PyTorch提供的数据加载器。

nltk：自然语言工具包（Natural Language Toolkit），提供了各种文本处理和分析的功能。

pandas：用于处理数据的Python库。

load_dataset、load_metric：Hugging Face开源的数据集和评估指标库。

transformers：用于自然语言处理和深度学习模型的Python库，包括预训练的语言模型（如BERT、GPT等）和用于文本分类、命名实体识别、问答等任务的工具。

AutoConfig、BartForConditionalGeneration、DataCollatorForSeq2Seq、HfArgumentParser、Seq2SeqTrainer、Seq2SeqTrainingArguments、TapexTokenizer：用于训练和生成seq2seq模型的相关类。

set_seed、get_last_checkpoint、is_main_process、default_compute_objective：用于设置训练过程中的一些参数和函数。

os.environ：用于设置环境变量的Python模块。

总体而言，这段代码主要是为后续的seq2seq模型训练准备环境和依赖项。其中设置的环境变量可能与多线程/多进程相关，可以在后续的程序运行中提高效率。

In [7]:
def seed_everything(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    set_seed(seed)
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

seed_everything(42)

定义了一个函数seed_everything()，用于设置随机数种子，以保证程序的可重复性。具体来说，该函数接受一个整数seed作为参数，并使用该种子值调用np.random.seed()、torch.manual_seed()、set_seed()等函数，以在多个随机数生成操作中使用相同的种子值。

此外，函数还使用torch.cuda.is_available()判断当前是否有可用的GPU，如果有则继续调用torch.cuda.manual_seed()设置GPU的随机数种子，并将torch.backends.cudnn.deterministic设置为True，以保证在GPU上进行的计算结果可重复。

最后，函数调用seed_everything(42)，将种子值设置为42，以在后续的程序执行中保证随机数的可重复性。

In [8]:
#data = pd.read_csv('/kaggle/input/policydata/policy_20231102170949.csv')

In [9]:
# import pandas as pd

# # 读取数据集
# data = pd.read_csv('/kaggle/input/policydata/policy_20231102170949.csv')

# # 统计每列的空值数量
# null_counts = data.isnull().sum()

# # 统计总共有多少行包含空值
# rows_with_null = data.isnull().any(axis=1).sum()

# # 输出每列的空值数量
# print("每列的空值数量：")
# print(null_counts)

# # 输出包含空值的行数
# print("包含空值的行数：", rows_with_null)

查看包含空值的列

使用了datasets库中的load_dataset()函数，用于加载数据集。具体而言，它从json文件中加载了三个数据集，分别是训练集、验证集和测试集。

load_dataset()函数的第一个参数是数据集名称，这里使用了'json'。第二个参数data_files是一个字典，指定了各个数据集对应的文件路径，其中'train'对应训练集文件路径，'valid'对应验证集文件路径，'test'对应测试集文件路径。这里的文件路径分别为'/kaggle/input/tableqa/fetaQA-v1_train.json'、'/kaggle/input/tableqa/fetaQA-v1_dev.json'和'/kaggle/input/tableqa/fetaQA-v1_test.json'。

最后一个参数field='data'指定了数据集中的字段名，这里使用了'data'作为字段名。

通过上述代码，dataset变量将包含加载的数据集，可以通过dataset['train']、dataset['valid']和dataset['test']来访问对应的训练集、验证集和测试集数据。

In [10]:
#data

In [11]:
config = AutoConfig.from_pretrained('microsoft/tapex-base')

config.no_repeat_ngram_size = 0
config.max_length = 1024
config.early_stopping = False

# load tapex tokenizer
tokenizer = TapexTokenizer.from_pretrained(#使用的是特殊的TapexTokenizer
    'microsoft/tapex-base',
    use_fast=True,
    add_prefix_space=True
)

# load Bart based Tapex model (default tapex-large)
model = BartForConditionalGeneration.from_pretrained('microsoft/tapex-base', config=config)

使用了Hugging Face的transformers库，通过调用相应的函数从预训练模型中加载配置信息、分词器和模型。

TAPex模型基于BART架构，这是一种基于Transformer的序列到序列模型，同时具备自编码器和生成模型的能力。通过在大规模的代码库和自然语言文本数据上进行预训练，TAPex模型能够学习到代码和自然语言之间的关联信息。

首先，使用AutoConfig.from_pretrained()函数从预训练模型'microsoft/tapex-base'加载配置信息，并将其存储在config变量中。no_repeat_ngram_size被设置为0，表示生成的文本中不会重复ngram。max_length被设置为1024，表示生成的文本的最大长度。early_stopping被设置为False，表示不启用提前停止策略。

接下来，使用TapexTokenizer.from_pretrained()函数从预训练模型'microsoft/tapex-base'加载分词器，并将其存储在tokenizer变量中。use_fast=True表示使用快速分词器，add_prefix_space=True表示在分词时在句子前添加空格，以处理特定的语言模型需求。

最后，使用BartForConditionalGeneration.from_pretrained()函数从预训练模型'microsoft/tapex-base'加载Bart based Tapex模型，并将其存储在model变量中。同时，将之前加载的配置信息config传递给模型。

通过上述代码，我们可以使用tokenizer对文本进行分词，并使用model进行文本生成或其他相关任务。

In [12]:
# # 另一个模型
# config = AutoConfig.from_pretrained("t5-base")
# tokenizer = AutoTokenizer.from_pretrained("t5-base")
# model = AutoModelForSeq2SeqLM.from_pretrained(
#     "t5-base",
#     from_tf=False,
#     config=config,
# )

In [15]:
import pandas as pd

# 从原始文件读取表格数据
df = pd.read_csv("/kaggle/input/policydataset/policy_20231102170949.csv")

# 提取前100行数据
df_head = df.head(100)

# 将提取的数据保存到另一个文件
df_head.to_csv("/kaggle/working/policy_100.csv", index=False)

这个是为了将数据集缩小，用于先跑通代码

In [17]:
import pandas as pd

# 读取CSV文件并转换为DataFrame
data = pd.read_csv('/kaggle/working/policy_100.csv')#要进行真正训练时需要修改

def preprocess_examples(examples):
    questions = examples['title']
    tables = pd.DataFrame({
#         'id': examples['id'],
        'city': examples['city'],
        'title': examples['title'],
        #'sub_title': examples['sub_title'],
        #'pub_no': examples['pub_no'],
        'pub_date': examples['pub_date'],
        #'effect_date': examples['effect_date']
    })
    answers = examples['content']  # 将"content"列作为"answer"

    model_inputs = tokenizer(
        table=tables, query=questions, max_length=1024, padding="max_length", truncation=True
    )
    labels = tokenizer(
        answer=answers,
        max_length=64,
        padding="max_length",
        truncation=True,
    )
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]
    
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs



上面的data里的没有question列，尝试将问题列改为title列，并且table中有些数据是空值，现在是将这些有空值的列没有进行考虑

实现了一个名为preprocess_examples的函数，用于将给定的输入数据进行预处理以供模型训练。以下是代码实现的主要步骤：

将问题文本转换为小写字母，并将其保存在名为questions的列表中。（但是文本中并没有问题文本，用title列进行替代）

提取每个例子中的表格数据，并将其转换为pandas DataFrame格式。将所有数据表保存在名为tables的列表中。（table表格中也对一些具有空值的数进行了不考虑处理）

从输入数据中提取答案，并将其保存在名为answers的列表中。

通过调用一个名为tokenizer的分词器，将表格数据和问题文本编码为模型的输入。同时，使用padding和truncation参数来确保所有模型输入都是相同的长度，并将结果保存在名为model_inputs的字典中。

使用tokenizer再次对答案进行编码，并将结果保存在名为labels的字典中。同时，通过将标签中的pad_token_id替换为-100，创建一个mask以指示哪些元素是真实的标签（值为1）和哪些元素是填充的（值为0）。

最后，将labels中的input_ids赋值给model_inputs中的labels，以便在模型训练时使用。

总之，该函数将输入数据预处理为模型的输入和输出，并返回一个包含这些数据的字典。

In [ ]:
# print(data.columns)

In [18]:
from datasets import Dataset
# 对CSV中的数据进行预处理
#encoded_train_ds = preprocess_examples(data)
# 如果需要转换成Hugging Face Datasets格式，可以使用以下代码
encoded_train_ds = Dataset.from_pandas(data)  # 将DataFrame转换为Hugging Face Datasets格式
encoded_train_ds = encoded_train_ds.map(preprocess_examples, batched=True, remove_columns=encoded_train_ds.column_names)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


  0%|          | 0/1 [00:00<?, ?ba/s]

使用了datasets库中的map()函数，用于将原始数据集中的每个样本映射到经过处理的样本上，并返回一个新的数据集。具体而言，它使用preprocess_examples()函数对训练集、验证集和测试集进行了预处理，将原始数据集转换为模型需要的输入格式，并将处理后的结果存储在encoded_train_ds、encoded_val_ds和encoded_test_ds变量中。（目前是将数据集转换为训练集，结果存储在encoded_train_ds里）

map()函数的第一个参数是一个函数，用于对原始数据集中的每个样本进行处理。这里使用了preprocess_examples作为处理函数。

batched=True表示对数据集进行批处理，默认情况下，map()函数会处理单个样本，但启用批处理可以提高处理效率。

remove_columns参数指定要从数据集中删除的列，这里我们删除了原始数据集中的所有列，只保留了处理后的列。

通过上述代码，我们可以将原始数据集转换为模型需要的输入格式，并将其存储在新的数据集中，以便后续训练和评估。


In [19]:
tokenizer.decode(encoded_train_ds[0]['input_ids'])

'<s> 宁夏回族自治区人民政府立法工作规定 col : city | title | pub_date row 1 : nx |  宁夏回族自治 | 2022-12-23 row 2 : nx |  宁夏回族自治 | 2022-9-30 row 3 : nx |  宁夏回族自治 | 2022-1-8 row 4 : nx |  宁夏回族自治 | 2021-7-29 row 5 : nx |  宁夏回族自治 | 2020-12-22 row 6 : nx |  宁夏回族自治 | 2020-12-22 row 7 : nx |  宁夏回族自治 | 2020-3-18 row 8 : nx |  宁夏回族自治 | 2020-1-9 row 9 : nx |  宁夏回族自治 | 2020-1-9 row 10 : nx |  宁夏回族自治 | 2019-12-31 row 11 : nx |  宁夏回族自治 | 2019-5-8 row 12 : nx |  宁夏回族自治 | 2019-3-2 row 13 : nx |  宁夏回族自治 | 2018-12-24 row 14 : nx |  宁夏回族自治 | 2018-12-24 row 15 : nx |  宁夏回族自治 | 2018-12-24 row 16 : nx |  宁夏回族自治 | 2018-9-4 row 17 : nx |  宁夏回族自治 | 2018-1-4 row 18 : nx |  宁夏回族自治 | 2018-1-4 row 19 : nx |  宁夏回族自治 | 2017-12-21 row 20 : nx |  宁夏回族自治 | 2017-12-21 row 21 : nx |  宁夏回族自治 | 2017-11-30 row 22 : nx |  宁夏回族自治 | 2017-11-17 row 23 : nx |  宁夏回族自治 | 2017-1-20 row 24 : nx |  宁夏回族自治 | 2017-1-12 row 25 : nx |  宁夏回族自治 | 2016-12-23 row 26 : nx |  宁夏回族自治 | 2016-9-2 row 27 : nx |  宁夏回族自治 | 2016-8-17 row 28 : nx |  宁夏回族自治 | 2016-7-12 r

使用了之前加载的分词器tokenizer，通过调用decode()函数将经过编码后的样本数据转换回文本格式。

具体而言，它对encoded_train_ds[0]['input_ids']这个已编码的样本数据进行解码，将其转换为原始文本格式。encoded_train_ds[0]表示训练集中的第一个样本，['input_ids']表示该样本的编码输入部分。

通过查看这个解码后的文本，我们可以更好地理解模型处理数据的方式，并检查预处理和编码是否正确。

In [20]:
tokenizer.decode([x for x in encoded_train_ds[0]['labels'] if x != -100])

'<s> 宁夏回族自治区人民政府立法工作规定\n2022年12月23日宁夏回族自治</s>'

encoded_train_ds[0]['labels']是一个包含标签的列表。在这里，我们使用了列表推导式来过滤掉值为-100的标签，并将剩余的标签传递给tokenizer.decode()函数进行解码。

tokenizer.decode()函数将经过过滤的标签列表作为输入，将其转换为原始文本格式。

通过执行这段代码，我们可以查看第一个训练样本的标签在解码后的文本表示中的内容。这对于理解标签和模型输出之间的对应关系很有帮助。

In [21]:
import json
def save_json(content, path, indent=4, **json_dump_kwargs):
    with open(path, "w") as f:
        json.dump(content, f, indent=indent, sort_keys=True, **json_dump_kwargs)

定义了一个名为save_json()的函数，用于将Python对象保存为JSON格式的文件。

函数接受三个参数：

content：要保存为JSON的Python对象。
path：要保存JSON文件的路径。
indent：可选参数，表示JSON文件的缩进空格数，默认为4。
**json_dump_kwargs：可选参数，用于传递给json.dump()函数的其他关键字参数。
在函数内部，使用open()函数以写入模式打开指定路径的文件，并将文件对象赋值给变量f。然后，使用json.dump()函数将Python对象content转换为JSON格式，并将结果写入文件中。indent参数指定缩进空格数，sort_keys=True表示按照键的字典序对JSON进行排序。**json_dump_kwargs用于传递其他关键字参数，如果有的话。

通过调用save_json()函数，你可以将Python对象保存为JSON文件，可以根据需要调整缩进、排序和其他相关的JSON保存选项。

这个只定义了一下函数，具体在下文才用上

In [22]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
)

定义了一个名为DataCollatorForSeq2Seq的数据收集器，用于将多个序列样本组合成批次，并进行填充和截断以满足模型的需求。

DataCollatorForSeq2Seq接受三个参数：

tokenizer：用于将原始文本转换为模型需要的输入格式。
model：用于确定最大输入长度的模型实例。
label_pad_token_id：标签数据集中要用作填充令牌的ID。
在这里，我们使用DataCollatorForSeq2Seq来处理序列到序列模型的训练数据。通过指定上述三个参数，数据收集器将利用分词器tokenizer将原始文本转换为模型需要的输入格式，并使用模型实例model确定最大输入长度。同时，由于这是一个序列到序列模型，我们还需要指定标签数据集中用作填充令牌的ID。

通过调用DataCollatorForSeq2Seq函数，我们可以有效地处理多个序列样本，并以批量方式提供给模型进行训练。

In [23]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=250c8d2df62ef189c5736494b0ffecd8181d718a244833d5fdffb84ba6d47e4c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [24]:
from datasets import load_metric
import nltk

# 加载 ROUGE metric
rouge_metric = load_metric("rouge")

# 根据模型输出和真实标签进行文本后处理
def postprocess_text(preds, labels, metric_name):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # 对于 ROUGE 指标，需要在每个句子后面增加换行符
    if metric_name == "rouge":
        preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
        labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]
    elif metric_name == "sacrebleu":  # sacrebleu 指标
        labels = [[label] for label in labels]
    elif metric_name == "bleu":
        preds = [pred.split(' ') for pred in preds]
        labels = [[label.split(' ')] for label in labels]
    else:
        pass

    return preds, labels

定义了一个名为postprocess_text()的后处理函数，用于对模型生成的文本进行一些额外的处理，以便与参考标签进行比较。

函数接受三个参数：

preds：模型生成的文本列表。
labels：参考标签的文本列表。
metric_name：评估指标的名称，可以是"rouge"、"sacrebleu"或"bleu"。
在函数内部，首先使用列表推导式对preds和labels中的文本进行去除首尾空格的操作，将其存储在preds和labels变量中。

然后，根据metric_name的值，进行不同的后处理操作：

如果metric_name是"rouge"，则使用NLTK库的sent_tokenize()函数将每个预测和参考标签的文本分割成句子，并使用换行符连接这些句子，存储在preds和labels变量中。
如果metric_name是"sacrebleu"，将参考标签的文本列表中的每个标签放入一个单独的列表中，存储在labels变量中。
如果metric_name是"bleu"，将每个预测文本按空格分割成单词列表，将参考标签的文本列表中的每个标签按空格分割成单词列表的列表，分别存储在preds和labels变量中。
如果metric_name不是以上三种情况，则不进行任何处理。
最后，返回经过后处理的preds和labels列表。

通过调用postprocess_text()函数，可以对模型生成的文本进行适当的后处理，以便与参考标签进行评估。根据所使用的评估指标，可以选择执行不同的后处理操作。

In [25]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/",
    overwrite_output_dir=False,
    do_train=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=10,
    weight_decay=0.01,
    predict_with_generate=True,
    learning_rate=0.1,
    optim='adamw_torch',
    save_strategy='epoch',
    evaluation_strategy='no',#之后改为no
    seed=42,
    report_to='none',
    #max_steps=500,  # 将值设置为 500
)

（训练时参数需要进行修改）

定义了一个名为Seq2SeqTrainingArguments的训练参数对象，用于配置Seq2Seq模型的训练过程。

对象接受多个参数：

output_dir：训练输出结果的目录路径。
overwrite_output_dir：如果output_dir已存在，是否覆盖。
do_train、do_eval、do_predict：是否分别执行训练、评估和预测。
num_train_epochs：训练轮数。
per_device_train_batch_size：每个GPU设备的训练批次大小。
per_device_eval_batch_size：每个GPU设备的评估批次大小。
warmup_steps：优化器增加学习率的热身步骤数。
weight_decay：优化器中权重衰减的比例。
predict_with_generate：是否使用生成模式进行预测。
learning_rate：优化器的学习率。
optim：优化器的类型。
save_strategy：保存检查点的策略。可以是steps（每隔一定步数保存一次）或epoch（每个 epoch 结束时保存）。
evaluation_strategy：评估策略。可以是steps（每隔一定步数评估一次）或epoch（每个 epoch 结束时评估）。
seed：随机数种子。
通过创建Seq2SeqTrainingArguments对象，可以灵活地配置Seq2Seq模型的训练过程。在实际的训练过程中，该对象将被传递给Seq2SeqTrainer类的构造函数，用于控制训练、评估和预测的行为。

In [26]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
      preds = preds[0]
    decoded_raw_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_raw_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    dic_pred_label = {'predictions': decoded_raw_preds, 'labels': decoded_raw_labels}
    save_json(dic_pred_label, os.path.join(training_args.output_dir, "detokenized_outputs.json"))

    result = {}
    # Only compute 'bertscore' metric
    metric_name = "bertscore"
    metric = load_metric(metric_name)
    decoded_preds, decoded_labels = postprocess_text(decoded_raw_preds, decoded_raw_labels, metric_name)
    res = metric.compute(predictions=decoded_preds, references=decoded_labels, lang="en")
    for k,v in res.items():
      if k =="hashcode":
        continue
      result[f"{metric_name}_{k}_0"] = round(v[0], 2)
      result[f"{metric_name}_{k}_1"] = round(v[1], 2)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


定义了一个名为compute_metrics()的函数，用于计算Seq2Seq模型评估指标。

函数接受一个参数eval_preds，表示模型的预测结果和对应的标签。其中preds表示预测结果，labels表示对应的标签。函数首先使用batch_decode()方法将preds和labels转换为原始文本，并将其保存到一个字典中。该字典中包含两个键 predictions 和 labels，分别对应于预测的文本和真实标签的文本。

接下来，函数对原始文本进行一些简单的后处理，以获取多个评估指标，如meteor、bleu、sacrebleu、bertscore和bleurt。对于每个指标，函数使用相应的load_metric()方法加载指标，并调用compute()方法计算预测结果和标签之间的指标值。对于bertscore指标，函数还需要指定语言类型（lang="en"）。

最后，函数计算生成文本的平均长度，并将所有指标值舍入到4位小数。返回值是一个字典，包含各个指标的名称和值。

In [27]:
 from sklearn.model_selection import train_test_split

# # 划分训练数据集和验证数据集
#encoded_train_ds, eval_train_ds = train_test_split(encoded_train_ds, test_size=0.1, random_state=42)
# #划分一小部分验证集，用来改正错误，但是验证集依然没有被使用
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_ds,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

创建了一个名为trainer的Seq2SeqTrainer对象，用于训练和评估Seq2Seq模型。

Seq2SeqTrainer类接受多个参数：

model：要训练的Seq2Seq模型。
args：包含训练参数的对象，通常是Seq2SeqTrainingArguments类型的对象。
train_dataset：用于训练的数据集，通常是经过编码处理后的训练数据集。
eval_dataset：用于评估的数据集，通常是经过编码处理后的验证数据集。
tokenizer：用于将原始文本转换为模型输入的分词器。
data_collator：用于将样本批次组合成适合模型输入的数据集处理器。
通过创建Seq2SeqTrainer对象，可以方便地配置并执行Seq2Seq模型的训练和评估过程。调用trainer.train()方法可以开始训练过程，调用trainer.evaluate()方法可以对模型进行评估，并根据需要保存模型的检查点和结果输出。

In [ ]:
# last_checkpoint = get_last_checkpoint(training_args.output_dir)
# print(last_checkpoint)

首先调用自定义函数get_last_checkpoint()，并传入训练参数对象training_args.output_dir作为参数。training_args.output_dir表示输出目录的路径，通常是存储训练过程中生成的检查点文件和其他输出文件的位置。

get_last_checkpoint()函数的作用是在指定的输出目录中查找以"checkpoint-"开头的文件，并返回最新的检查点文件的完整路径。如果找不到符合条件的检查点文件，则返回None。

接下来，将获取到的最新检查点文件的路径存储在变量last_checkpoint中。

最后，通过使用print()函数将last_checkpoint打印出来，以显示最新的检查点文件路径。这样可以方便查看最新的检查点文件在哪个位置。

In [ ]:
# dataset_length = len(encoded_train_ds)
# print("Dataset length:", dataset_length)

In [ ]:
# index = 1
# if index >= dataset_length:
#     print("Index is out of range.")

In [28]:
last_checkpoint = get_last_checkpoint(training_args.output_dir)
print(last_checkpoint)

all_metrics = {}

if training_args.do_train:
    if last_checkpoint is not None:
        checkpoint = last_checkpoint
    elif os.path.isdir("tapex-base"):
        checkpoint = "tapex-base"
    else:
        checkpoint = None

    if checkpoint is None:
        # 处理键不存在的情况，执行模型训练
        train_result = trainer.train()
        trainer.save_model()  # 保存模型和分词器

        metrics = train_result.metrics
        max_train_samples = len(encoded_train_ds)
        metrics["train_samples"] = min(max_train_samples, len(encoded_train_ds))

        if trainer.is_world_process_zero():
            metrics_formatted = trainer.metrics_format(metrics)
            print("***** train metrics *****")
            k_width = max(len(str(x)) for x in metrics_formatted.keys())
            v_width = max(len(str(x)) for x in metrics_formatted.values())
            for key in sorted(metrics_formatted.keys()):
                print(f"  {key: <{k_width}} = {metrics_formatted[key]:>{v_width}}")
            save_json(metrics, os.path.join(training_args.output_dir, "train_results.json"))
            all_metrics.update(metrics)

            # 需要保存训练状态，因为Trainer.save_model只会保存模型和分词器
            trainer.state.save_to_json(os.path.join(training_args.output_dir, "trainer_state.json"))
    else:
        # 处理键存在的情况，执行恢复训练
        train_result = trainer.train(resume_from_checkpoint=checkpoint)
        trainer.save_model()  # 保存模型和分词器

        metrics = train_result.metrics
        max_train_samples = len(encoded_train_ds)
        metrics["train_samples"] = min(max_train_samples, len(encoded_train_ds))

        if trainer.is_world_process_zero():
            metrics_formatted = trainer.metrics_format(metrics)
            print("***** train metrics *****")
            k_width = max(len(str(x)) for x in metrics_formatted.keys())
            v_width = max(len(str(x)) for x in metrics_formatted.values())
            for key in sorted(metrics_formatted.keys()):
                print(f"  {key: <{k_width}} = {metrics_formatted[key]:>{v_width}}")
            save_json(metrics, os.path.join(training_args.output_dir, "train_results.json"))
            all_metrics.update(metrics)
else:
    checkpoint = None  # 处理 training_args.do_train 为 False 的情况，将 checkpoint 设置为 None

None


Step,Training Loss


***** train metrics *****
  epoch                    =        3.0
  total_flos               =   170358GF
  train_loss               =    39.7716
  train_runtime            = 0:00:37.38
  train_samples            =        100
  train_samples_per_second =      8.025
  train_steps_per_second   =      2.006


主要是针对训练阶段的处理。首先，创建一个空字典all_metrics，用于存储所有的指标。

接下来，根据训练参数training_args中的设定，判断是否需要进行训练。如果training_args.do_train为True，则进入训练流程。在训练流程中，首先判断最新的检查点文件路径last_checkpoint是否存在。如果存在，则将该路径赋值给变量checkpoint；如果不存在，则判断是否存在名为"tapex-base"的目录，如果存在，则将"tapex-base"赋值给checkpoint，否则将其设置为None。

随后，调用trainer.train()方法开始训练过程。通过传递resume_from_checkpoint=checkpoint参数，可以在上次训练的检查点处继续训练。训练完成后，使用trainer.save_model()保存模型及其相关的分词器，以便后续上传等操作。

接下来，从训练结果train_result中获取指标信息，并将最大训练样本数max_train_samples设置为编码后训练数据集的长度。然后，将指标train_samples设置为实际训练样本数和max_train_samples中较小的值。

如果当前进程为主进程（trainer.is_world_process_zero()为True），则将指标结果进行格式化，并打印出来。然后，将指标保存为JSON文件并更新到all_metrics字典中。

最后，为了保存完整的训练状态，还需要将trainer.state保存为JSON文件，文件路径为os.path.join(training_args.output_dir, "trainer_state.json")。这样做可以在需要恢复训练时，使用TrainerState.load_from_json()方法加载训练状态。

In [ ]:
# last_checkpoint = get_last_checkpoint(training_args.output_dir)
# checkpoint = last_checkpoint

先调用自定义函数get_last_checkpoint()，并传入训练参数对象training_args.output_dir作为参数，从输出目录中获取最新的检查点文件路径，将其赋值给变量last_checkpoint。

接下来，将last_checkpoint赋值给变量checkpoint，以便在训练过程中使用该检查点文件路径。

In [ ]:
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=encoded_train_ds,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics
# )

创建了一个Seq2SeqTrainer对象trainer，用于训练序列到序列（seq2seq）模型。

构造函数的参数包括：

model：需要训练的模型。
args：训练参数对象。
train_dataset：经过编码后的训练数据集。
eval_dataset：经过编码后的验证数据集。
tokenizer：用于对文本进行分词和编码的分词器。
data_collator：用于处理训练数据的数据收集器。
compute_metrics：用于计算评估指标的函数。
通过这些参数，Seq2SeqTrainer对象可以管理整个训练过程，并提供了训练、评估、保存模型等功能。

In [ ]:
# trainer.train(resume_from_checkpoint=checkpoint)

调用trainer.train()方法开始训练过程。通过传递resume_from_checkpoint=checkpoint参数，可以在上次训练的检查点处继续训练。

在训练过程中，trainer对象将根据训练参数中指定的训练步数、学习率等进行参数更新，并计算损失、评估指标等信息。训练过程中会自动进行反向传播、梯度更新等操作，直到达到指定的训练步数或满足其他停止条件。

请注意，这里的checkpoint是之前获取到的最新检查点文件路径，用于恢复上次训练的状态。如果没有找到有效的检查点文件路径，则会从头开始训练。

In [29]:
# 定义保存路径
output_model_dir = "/path/to/save/model"

# 保存模型和tokenizer
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

('/path/to/save/model/tokenizer_config.json',
 '/path/to/save/model/special_tokens_map.json',
 '/path/to/save/model/vocab.json',
 '/path/to/save/model/merges.txt',
 '/path/to/save/model/added_tokens.json')

In [ ]:
model

In [ ]:
# results = {}
# if training_args.do_eval:
#   print(("*** Evaluate ***"))
#   metrics = trainer.evaluate(
#       max_length=64, 
#       num_beams=None, 
#       metric_key_prefix="eval"
#   )
#   max_val_samples = len(encoded_val_ds)
#   metrics["eval_samples"] = min(max_val_samples, len(encoded_val_ds))
  
#   if trainer.is_world_process_zero():
#     metrics_formatted = trainer.metrics_format(metrics)
#     print("***** val metrics *****")
#     k_width = max(len(str(x)) for x in metrics_formatted.keys())
#     v_width = max(len(str(x)) for x in metrics_formatted.values())
#     for key in sorted(metrics_formatted.keys()):
#       print(f"  {key: <{k_width}} = {metrics_formatted[key]:>{v_width}}")
#     save_json(metrics, os.path.join(training_args.output_dir, "eval_results.json"))
#     all_metrics.update(metrics)


用于在验证集上评估训练好的模型，并记录评估指标值。具体流程如下：

首先，判断是否需要进行评估（training_args.do_eval为True）。如果需要，则调用trainer.evaluate()方法，在验证集encoded_val_ds上对当前模型进行评估。

evaluate()方法的参数包括：

max_length：生成的文本最大长度。
num_beams：束搜索（beam search）过程中保留的翻译序列数量。如果为None或小于等于1，则使用贪心搜索（greedy search）。
metric_key_prefix：评估指标的前缀，用于区分不同数据集的指标。
评估完成后，从评估结果metrics中获取评估指标信息，并将最大验证样本数max_val_samples设置为编码后验证数据集的长度。然后，将指标eval_samples设置为实际验证样本数和max_val_samples中较小的值。

如果当前进程为主进程（trainer.is_world_process_zero()为True），则将指标结果进行格式化，并打印出来。然后，将指标保存为JSON文件并更新到all_metrics字典中。最后，将当前评估指标结果metrics保存在results字典中。

In [ ]:
# if training_args.do_predict:
#   print(("*** Test ***"))
#   test_results = trainer.predict(
#       encoded_test_ds,
#       metric_key_prefix="test",
#       max_length=64,
#       num_beams=None,
#     )
#   metrics = test_results.metrics
#   max_test_samples = len(encoded_test_ds)
#   metrics["test_samples"] = min(max_test_samples, len(encoded_test_ds))
    
#   if trainer.is_world_process_zero():
#     metrics_formatted = trainer.metrics_format(metrics)
#     print(("***** test metrics *****"))
#     k_width = max(len(str(x)) for x in metrics_formatted.keys())
#     v_width = max(len(str(x)) for x in metrics_formatted.values())
#     for key in sorted(metrics_formatted.keys()):
#       print(f"  {key: <{k_width}} = {metrics_formatted[key]:>{v_width}}")
#     save_json(metrics, os.path.join(training_args.output_dir, "test_results.json"))
#     all_metrics.update(metrics)

用于在测试集上进行预测，并记录预测结果的评估指标值。具体流程如下：

首先，判断是否需要进行预测（training_args.do_predict为True）。如果需要，则调用trainer.predict()方法，在测试集encoded_test_ds上对当前模型进行预测。

predict()方法的参数包括：

encoded_test_ds：编码后的测试数据集。
metric_key_prefix：评估指标的前缀，用于区分不同数据集的指标。
max_length：生成的文本最大长度。
num_beams：束搜索（beam search）过程中保留的翻译序列数量。如果为None或小于等于1，则使用贪心搜索（greedy search）。
预测完成后，从预测结果test_results中获取评估指标信息，并将最大测试样本数max_test_samples设置为编码后测试数据集的长度。然后，将指标test_samples设置为实际测试样本数和max_test_samples中较小的值。

如果当前进程为主进程（trainer.is_world_process_zero()为True），则将指标结果进行格式化，并打印出来。然后，将指标保存为JSON文件并更新到all_metrics字典中。

最后，将当前预测结果的评估指标结果metrics保存在results字典中，并进行更新。

In [ ]:
# from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# # 加载已保存的模型和tokenizer
# output_model_dir = "/kaggle/working/"
# model = AutoModelForQuestionAnswering.from_pretrained(output_model_dir)
# tokenizer = AutoTokenizer.from_pretrained(output_model_dir)


在做下游任务，利用WebQA问答对做数据集，来训练问答系统

In [ ]:
# # 读取训练集数据
# with open('/kaggle/input/dataset/WebQA.v1.0/me_train.json', 'r', encoding='utf-8') as file:
#     training_set = json.load(file)

# # 创建问题和答案列表
# questions = []
# answers = []

# # 遍历训练集中的每个问题
# for question_id, question_data in training_set.items():
#     question = question_data['question']
#     evidences = question_data['evidences']
    
#     # 遍历每个证据项，将答案和问题添加到列表中
#     for evidence_id, evidence_data in evidences.items():
#         answer = evidence_data['answer'][0]
#         evidence_text = evidence_data['evidence']
        
#         questions.append(question)
#         answers.append(answer)


In [ ]:
import json

# 读取JSON文件
with open('/kaggle/input/webqadataset/WebQA.json', 'r') as file:
    data = json.load(file)

# 计算要提取的问答对数量
total_items = len(data)
extract_count = max(total_items // 10000, 1)   # 只提取总数的万分之一（至少提取一个）

# 提取数据
extracted_data = [{"question": item["question"], "id": item["id"], "answer": item["passages"][0]["answer"]} for i, item in enumerate(data) if i % extract_count == 0]

# 将提取的数据保存为新的JSON文件
with open('/kaggle/working/WebQA_Extracted.json', 'w') as file:
    json.dump(extracted_data, file)


In [ ]:
import json

# 读取JSON文件
with open('/kaggle/input/dataset/WebQA.v1.0/me_train.json', 'r') as file:
    data = json.load(file)

# 计算要提取的数据量
total_data = len(data)
extracted_data = total_data // 10000

# 提取数据
keys = list(data.keys())[:extracted_data]
extracted_data = {key: data[key] for key in keys}

# 将提取的数据保存为新的JSON文件
with open('/kaggle/working/me_train_10000.json', 'w') as file:
    json.dump(extracted_data, file)


In [ ]:
print(type(data))


In [54]:
# import json
# from transformers import AutoModelForQuestionAnswering, AutoTokenizer
# from torch.utils.data import DataLoader, Dataset
# import torch
# import torch.nn as nn
# import torch.optim as optim

# # 加载已保存的模型和tokenizer
# output_model_dir = "/kaggle/working/"
# model = AutoModelForQuestionAnswering.from_pretrained(output_model_dir)
# tokenizer = AutoTokenizer.from_pretrained(output_model_dir)

# import pandas as pd
# from torch.utils.data import Dataset

# class DownstreamDataset(Dataset):
#     def __init__(self, data_file, tokenizer):
#         try:
#             with open(data_file, 'r', encoding='utf-8') as f:
#                 data = json.load(f)
#                 self.qas_data = pd.DataFrame(data)['qas']  # 获取'qas'列
#                 self.tokenizer = tokenizer
#         except Exception as e:
#             print("An error occurred while loading the dataset:", e)

#     def __len__(self):
#         return len(self.qas_data)

#     def __getitem__(self, index):
#         qas_list = self.qas_data[index]  # 获取列表对象
#         qas_json = json.dumps(qas_list)  # 将列表对象转换为JSON字符串
#         qas_dict = json.loads(qas_json)  # 解析JSON字符串为Python对象
#         question = str(qas_dict[0]['query_text'])  # 获取问题文本
#         answer = str(qas_dict[0].get('answer', ''))  # 获取答案文本，如果不存在则返回空字符串
#         encoding = self.tokenizer.encode_plus(
#             question,
#             answer,
#             # 其他参数
#             return_tensors='pt'
#         )
#         return {
#             'input_ids': encoding['input_ids'].flatten(),
#             'attention_mask': encoding['attention_mask'].flatten(),
#             # 其他返回值
#         }


# print(1)
# # 创建下游任务的数据集
# downstream_dataset = DownstreamDataset('/kaggle/input/dataset2/cmrc2018_train.json', tokenizer)
# print(1.5)
# # 创建数据加载器
# batch_size = 3
# data_loader = DataLoader(downstream_dataset, batch_size=batch_size, shuffle=True)
# print(len(data_loader))
# # 定义损失函数和优化器
# loss_fn = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
#     # 将模型设置为训练模式
# model.train()
# print(2)
# # 开始下游任务的训练
# num_epochs = 10
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)
# print(device)
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     print(len(data_loader))
#     for batch_idx, batch in enumerate(data_loader):
#         print(len(data_loader))
#         batch_input_ids = batch['input_ids'].to(device)
#         batch_attention_mask = batch['attention_mask'].to(device)

#         optimizer.zero_grad()

#         outputs = model(
#             input_ids=batch_input_ids,
#             attention_mask=batch_attention_mask
#         )

#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#         if (batch_idx + 1) % 10 == 0:  # 每处理10个batch打印一次信息
#             print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(data_loader)}], Loss: {loss.item():.4f}")

#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(data_loader)}")
# print(3)


# # 训练完成后，你可以保存或使用训练好的模型进行预测或评估

Some weights of BartForQuestionAnswering were not initialized from the model checkpoint at /kaggle/working/ and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1
1.5
801
2
cuda
801


AttributeError: 'str' object has no attribute 'empty'

In [60]:
import json
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# 加载已保存的模型和tokenizer
output_model_dir = "/kaggle/working/"
model = AutoModelForQuestionAnswering.from_pretrained(output_model_dir)
tokenizer = AutoTokenizer.from_pretrained(output_model_dir)

class DownstreamDataset(Dataset):
    def __init__(self, data_file, tokenizer):
        try:
            with open(data_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                self.qas_data = pd.DataFrame(data)['qas']  # 获取'qas'列
                self.tokenizer = tokenizer
        except Exception as e:
            print("An error occurred while loading the dataset:", e)

    def __len__(self):
        return len(self.qas_data)

    def __getitem__(self, index):
        qas_list = self.qas_data[index]  # 获取列表对象
        qas_json = json.dumps(qas_list)  # 将列表对象转换为JSON字符串
        qas_dict = json.loads(qas_json)  # 解析JSON字符串为Python对象
        question = str(qas_dict[0]['query_text'])  # 获取问题文本
        answer = str(qas_dict[0].get('answer', ''))  # 获取答案文本，如果不存在则返回空字符串
        encoding = self.tokenizer.encode_plus(
            question,
            answer,
            add_special_tokens=True,
            padding='max_length',
            max_length=512,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
        }

# 创建下游任务的数据集
downstream_dataset = DownstreamDataset('/kaggle/input/dataset2/cmrc2018_train.json', tokenizer)

# 创建数据加载器
# 创建数据加载器
batch_size = 3
data_loader = DataLoader(downstream_dataset, batch_size=batch_size, shuffle=True)

# 将DataLoader转换为迭代器
data_iter = iter(data_loader)

# 开始下游任务的训练
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx in range(len(data_loader)):  # 使用range(len(data_loader))遍历迭代器
        batch = next(data_iter)  # 获取下一个数据样本

        batch_input_ids = batch['input_ids'].to(device)
        batch_attention_mask = batch['attention_mask'].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=batch_input_ids,
            attention_mask=batch_attention_mask
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (batch_idx + 1) % 10 == 0:  # 每处理10个batch打印一次信息
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(data_loader)}], Loss: {loss.item():.4f}")

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(data_loader)}")



Some weights of BartForQuestionAnswering were not initialized from the model checkpoint at /kaggle/working/ and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'str' object has no attribute 'empty'

In [80]:
import json
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import pandas as pd

# 加载已保存的模型和tokenizer
output_model_dir = "/kaggle/working/"
model = AutoModelForQuestionAnswering.from_pretrained(output_model_dir)
tokenizer = AutoTokenizer.from_pretrained(output_model_dir)
os.listdir('/kaggle/input/dataset2/')

class DownstreamDataset(Dataset):
    def __init__(self, data_file, tokenizer):
        try:
            with open(data_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                self.qas_data = pd.DataFrame(data)['qas']  # 获取'qas'列
                self.tokenizer = tokenizer
        except Exception as e:
            print("An error occurred while loading the dataset:", e)

    def __len__(self):
        return len(self.qas_data)

    def __getitem__(self, index):
        qas_list = self.qas_data[index]  # 获取列表对象
        qas_json = json.dumps(qas_list)  # 将列表对象转换为JSON字符串
        qas_dict = json.loads(qas_json)  # 解析JSON字符串为Python对象
        question = str(qas_dict[0]['query_text'])  # 获取问题文本
        answer = str(qas_dict[0].get('answer', ''))  # 获取答案文本，如果不存在则返回空字符串
        encoding = self.tokenizer.encode_plus(
            question,
            answer,
            add_special_tokens=True,
            padding='max_length',
            max_length=512,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
        }

# 创建下游任务的数据集
import os

file_path = '/kaggle/input/dataset2/cmrc2018_train.json'
if os.path.exists(file_path):
    print(f"The file {file_path} exists.")
else:
    print(f"The file {file_path} does not exist.")

downstream_dataset = DownstreamDataset('/kaggle/input/dataset2/cmrc2018_train.json', tokenizer)
print(type(downstream_dataset))
# 定义训练参数
training_args = TrainingArguments(
    output_dir=output_model_dir,  # 模型保存路径
    num_train_epochs=10,  # 训练轮数
    per_device_train_batch_size=3,  # 每个设备的训练batch size
    logging_dir='./logs',  # 训练日志保存路径
    logging_steps=10
)

# 定义Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downstream_dataset,
)
print(type(table))
print(1)
if table.empty:
    print("Table is empty")
else:
    print("非空")
# 开始训练
trainer.train()

Some weights of BartForQuestionAnswering were not initialized from the model checkpoint at /kaggle/working/ and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


The file /kaggle/input/dataset2/cmrc2018_train.json exists.
<class '__main__.DownstreamDataset'>
<class 'torch.utils.data.dataloader.DataLoader'>
1


AttributeError: 'DataLoader' object has no attribute 'empty'

In [57]:
type(data_loader)


torch.utils.data.dataloader.DataLoader

In [ ]:
print(downstream_dataset.head())

In [ ]:
downstream_dataset

In [ ]:
import pandas as pd

# 读取整个JSON文件
data = pd.read_json('/kaggle/input/dataset/WebQA.v1.0/me_train.json')

# 提取前100条数据
first_100_rows = data.head(1)

# 将前100条数据保存回JSON文件
first_100_rows.to_json('/kaggle/working/me_train_1.json', orient='records')

# 输出提示信息
print("已将文件保存为：/kaggle/working/me_train_100.json")


In [ ]:
import pandas as pd

# 读取整个JSON文件
data = pd.read_json('/kaggle/input/dataset/WebQA.v1.0/me_train.json')

# 提取前100条数据
first_100_rows = data.head(100)

# 打印提取的数据
print(first_100_rows)


In [ ]:
import pandas as pd

# 读取数据文件
data = pd.read_json('/kaggle/working/me_train_1000.json')

# 打印列名
# print(data.head())
print(data.columns)


In [ ]:
# 保存训练好的模型
save_model_dir = "/kaggle/working/model"
model.save_pretrained(save_model_dir)
tokenizer.save_pretrained(save_model_dir)

下面这个代码应该还需要进行修改

In [ ]:
# # 给定的问题
# question = "数据收益合理化分配的原则和收益分配机制分别是什么？"

# answer = model(question, content)["answer"]

# print(answer)

In [ ]:
# from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# # 加载已保存的模型和tokenizer
# output_model_dir = "/kaggle/working/"
# model = AutoModelForQuestionAnswering.from_pretrained(output_model_dir)
# tokenizer = AutoTokenizer.from_pretrained(output_model_dir)

# # 提取context列
# context = data['content']

# # 将列表元素连接为单个字符串
# text = "\n".join(context)
# import pandas as pd

# # 将文本字符串转换为表格对象
# text_df = pd.DataFrame([text], columns=['text'])

# question = "数据收益合理化分配的原则和收益分配机制分别是什么？"

# # 构建输入数据
# inputs = tokenizer.encode_plus(text_df, question, padding=True, truncation=True, return_tensors="pt")

# # 在模型上进行推理
# outputs = model(**inputs)

# # 获取答案
# answer_start = torch.argmax(outputs.start_logits)
# answer_end = torch.argmax(outputs.end_logits)
# answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end+1]))

# # 打印答案
# print(answer)

In [ ]:
#pip install --upgrade transformers

In [ ]:
# # import torch
# # from transformers import BartTokenizer

# # # 加载模型和分词器
# # model = BartForQuestionAnswering.from_pretrained("model_name")
# # tokenizer = BartTokenizer.from_pretrained("model_name")

# # 加载数据集
# #data = pd.read_csv("your_dataset.csv")  # 将 "your_dataset.csv" 替换为你的数据集文件名
# # 将表格数据合并为一个单独的 DataFrame 对象
# table_example = pd.concat(data, axis=0, ignore_index=True)
# # 定义问题
# question = "数据收益合理化分配的原则和收益分配机制分别是什么？"

# # 将表格示例转换为文本格式
# table_text = table_example.to_string(index=False)

# # 使用 tokenizer 进行问答
# inputs = tokenizer(question, table_text, truncation=True, padding=True, return_tensors="pt")
# outputs = model(**inputs)

# # 解码答案
# answer = tokenizer.decode(outputs.start_logits, inputs["attention_mask"])

# # 打印答案
# print(answer)

In [ ]:
# from transformers import BartTokenizer, BartForQuestionAnswering

# tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
# model = BartForQuestionAnswering.from_pretrained('facebook/bart-large')

question = "数据收益合理化分配的原则和收益分配机制分别是什么？"
inputs = tokenizer.encode_plus(question, return_tensors='pt')

input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

outputs = model(input_ids, attention_mask=attention_mask)

start_logits = outputs.start_logits
end_logits = outputs.end_logits

start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits)

answer_tokens = input_ids[0][start_index:end_index+1]
answer = tokenizer.decode(answer_tokens)
print(answer)

In [ ]:
import torch
from bert_score import score

# 假设answers是一个包含所有回答的列表，reference是正确答案
answers = ["第一个回答", "第二个回答", "第三个回答", ...]  # 替换为实际的回答列表
reference = "正确的答案"  # 替换为实际的正确答案

all_scores = []
for ans in answers:
    P, R, F1 = score([ans], [reference], lang="zh", rescale_with_baseline=True)
    all_scores.append(F1.item())

average_score = sum(all_scores) / len(all_scores)
print("每个回答的BERTScore：", all_scores)
print("平均BERTScore：", average_score)
